<a href="https://colab.research.google.com/github/Axle-Bucamp/Recomandation-System-Gan/blob/main/CalculDiversityOverTime_Optimized_recommandationGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Comparaison entre la diversité de recommandation du modèle par rapport à un aléatoire. Calcul de la perte de diversité dans l'historique de recommandation

Validation ou rejet de l'hypothèse de résolution des effets Bulles par rapport à un aléatoire (en cas de rejets des tests sur des utilisateurs sont nécessaire selon l'article de calcul des effets bulles). Si la diversité de recommandation et la diversité de l'historique d'un utilisateur est comparable à un aléatoire alors le biais est solvé, l'algorythme explore suffisament pour permettre à l'utilisateur de continuer de découvrir de nouveaux films si il en a l'envie. Sinon il est nécessaire afin de le résoudre d'accepter une perte de précision sauf si des tests directement liés aux utilisateur afirme une validation de l'hypothèse.

Pensez à télécharger votre version du modèle ici selon le scripte optimisé de l'entrainement : "Training_Optimized_recommandationGAN".

In [1]:
import h5py
from ipywidgets import widgets
import os
import numpy as np
import pandas as pd
import random

import tensorflow as tf
import pathlib
import progressbar
import datetime
import time

from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import pdist
from scipy.sparse import csr_matrix
from scipy.special import expit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from IPython import display
from tensorflow import keras

In [2]:
import os
import urllib.request
urllib.request.urlretrieve("https://files.grouplens.org/datasets/movielens/ml-25m.zip","ml-25m.zip")
!unzip "ml-25m.zip" -d "ml-25m"

Archive:  ml-25m.zip
   creating: ml-25m/ml-25m/
  inflating: ml-25m/ml-25m/tags.csv  
  inflating: ml-25m/ml-25m/links.csv  
  inflating: ml-25m/ml-25m/README.txt  
  inflating: ml-25m/ml-25m/ratings.csv  
  inflating: ml-25m/ml-25m/genome-tags.csv  
  inflating: ml-25m/ml-25m/genome-scores.csv  
  inflating: ml-25m/ml-25m/movies.csv  


In [3]:
user = pd.read_csv('ml-25m/ml-25m/ratings.csv')
user = user.sample(frac=1).reset_index(drop=True)
genome = pd.read_csv('ml-25m/ml-25m/genome-scores.csv')

In [4]:
genome.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [5]:
movie_list = np.unique(genome["movieId"]) #np.random.choice(np.unique(genome["movieId"]), size=10000, replace=False)
user = user[user['movieId'].isin(movie_list)].sort_values(by=['movieId'])
genome = genome[genome['movieId'].isin(movie_list)].sort_values(by=['movieId'])

In [6]:
pivot_genome = pd.pivot_table(genome, values='relevance', index=['tagId'],columns=['movieId'], aggfunc=np.sum)

In [7]:
user_row = list(sorted(user.userId.unique()))
movie_col = list(sorted(user.movieId.unique()))

data = user['rating'].tolist()
row = user.userId.astype('category').cat.codes
col = user.movieId.astype('category').cat.codes

table = csr_matrix((data, (row, col)), shape=(len(user_row), len(movie_col)))

nonzero_row_indice, _ = table.nonzero()
table =  table[np.unique(nonzero_row_indice)]

In [9]:
model = tf.keras.models.load_model("Generator.h5", compile=False)

In [10]:
UserRow = 128
ItemUserNorm = table

In [11]:
def rel_proximity(movieList):
    return np.mean(pdist(pivot_genome.iloc[:,movieList] , 'euclidean'))

In [12]:
np.seterr(invalid='ignore')
from scipy.spatial.distance import cdist

def create_input_matrix(choice,value):
  number_of_rows = table.shape[0]
  movielist = list(np.unique(user["movieId"]))
    
  ItemUserNorm[0,:] = 0
  ItemUserNorm[0,choice] = value
    
  dist_array = pairwise_distances(X=ItemUserNorm[0].toarray(), Y=ItemUserNorm, metric='euclidean')
  random_row = dist_array.reshape(number_of_rows).argsort()
  
  Xpred = ItemUserNorm[random_row[0:UserRow]]
    
  col =  np.array(np.sum(Xpred,axis=0).argsort()).flatten()[::-1]
  Xpred = Xpred[:,col[0:UserRow]].toarray()

  Xpred = Xpred.reshape(UserRow,UserRow,1) / np.max(Xpred)
  #Xpred = np.concatenate([Xpred, np.random.normal(0.5,0.5,UserRow*UserRow).reshape(UserRow,UserRow,1)], axis=2)

  return Xpred, col[0:UserRow]


In [13]:
def prediction_call(model, choice, value):
  xpred, rand_col = create_input_matrix(choice,value)
  ypred = model(tf.convert_to_tensor(xpred.reshape(1,UserRow,UserRow,xpred.shape[2]).astype(float))).numpy().reshape(UserRow,UserRow)
  max_indice = np.argpartition(ypred[0,:], -10)[-10:]
  return rand_col[max_indice[0]], random.randint(3, 5),rel_proximity(np.array(rand_col)[max_indice])


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import random
pd.options.mode.chained_assignment = None  # default='warn'
movielist = movie_list
# recommandé
distance_intra_reco = []
distance_reco = []

datasetSize = 30

bar = progressbar.ProgressBar(maxval=datasetSize, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

for y in range(30):
    choice = [movielist[random.randint(0, 6000)]]
    value = [random.randint(2, 5)]
    distance = []
    for i in range(10):
        new_choice,new_value,dist_intra = prediction_call(model,choice,value)
        distance_intra_reco.append(dist_intra)
        choice.append(new_choice)
        value.append(new_value)
        dist = rel_proximity(choice)
        distance.append(dist)
    distance_reco.append(distance)
    bar.update(y+1)

[========================================================================] 100%

In [16]:
#non recommandé
movieList = np.unique(user["movieId"])
distance_noreco = []

bar = progressbar.ProgressBar(maxval=datasetSize, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

for y in range(30):
    choice = [movielist[random.randint(0, 6000)]]
    value = [random.randint(2, 5)]
    distance = []
    for i in range(10):
        new_choice = random.choice( range(len(np.unique(user["movieId"])) ))
        new_value = random.randint(3, 5)
        choice.append(new_choice)
        value.append(new_value)
        dist = rel_proximity(choice)
        distance.append(dist)
    distance_noreco.append(distance) 
    bar.update(y+1)

[========================================================================] 100%

In [17]:
distance_intra_alea = []

bar = progressbar.ProgressBar(maxval=datasetSize, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

for y in range(30):
    
    for i in range(10):
        new_choice = []
        for z in range(10):
            new_choice.append( random.choice(range(len(np.unique(user["movieId"])) )) )
        distance_intra_alea.append(rel_proximity(new_choice))
        
    bar.update(y+1)

[========================================================================] 100%

In [18]:
from scipy.stats import ttest_ind
distance_reco = np.array(distance_reco)
distance_noreco = np.array(distance_noreco)
for i in range(10) :
    print("number of movie seen {} :".format(i))
    print(ttest_ind(distance_reco[:,i], distance_noreco[:,i]))


number of movie seen 0 :
Ttest_indResult(statistic=4.17199901195073, pvalue=0.00010230769269630922)
number of movie seen 1 :
Ttest_indResult(statistic=6.293777139079638, pvalue=4.4465453545000537e-08)
number of movie seen 2 :
Ttest_indResult(statistic=7.462348009775603, pvalue=4.89694167878957e-10)
number of movie seen 3 :
Ttest_indResult(statistic=8.261534750598166, pvalue=2.2354597708019497e-11)
number of movie seen 4 :
Ttest_indResult(statistic=9.73278574605006, pvalue=8.386686189149191e-14)
number of movie seen 5 :
Ttest_indResult(statistic=10.086583356052406, pvalue=2.2529978154797333e-14)
number of movie seen 6 :
Ttest_indResult(statistic=10.35818348312429, pvalue=8.288506702913971e-15)
number of movie seen 7 :
Ttest_indResult(statistic=9.915846912590505, pvalue=4.241604617884717e-14)
number of movie seen 8 :
Ttest_indResult(statistic=10.956993367339598, pvalue=9.414563173068525e-16)
number of movie seen 9 :
Ttest_indResult(statistic=10.926850424014775, pvalue=1.0493434991411194e

In [19]:
print("test répartition des distances intra groupes de recommandations")
print(ttest_ind(distance_intra_alea, distance_intra_reco))


test répartition des distances intra groupes de recommandations
Ttest_indResult(statistic=-38.56151782704477, pvalue=2.558817991695348e-164)
